<a href="https://colab.research.google.com/github/jorgelum/meuPortfolio/blob/master/OPU3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Unidade Curricular:** Operações Unitárias 3 \
**Coordenadora:** Profᵃ. Drᵃ. Gisele Atsuko Medeiros Hirata \
**Autor:** Jorge Luiz Mendes  

Bibliotecas Básicas

In [1]:
import pandas as pd # tratamento de dados/ tabela
import numpy as np # algebra linear

## Exercício 1

Uma mistura com 30 % molar de n-hexano, 32 % molar de n-heptano e 38 %
molar de n-octano é sujeita a uma destilação flash à pressão de 1,2 bar, em que
60 % da alimentação são vaporizados. Determine: \
a) A temperatura no vaso de flash (R:T = 106°C). \
b) As composições do vapor e líquido produzidos \
(R: x<sub>n-hexano</sub> = 13,7 % molar;  
x<sub>n-heptano</sub> = 31,1 % molar; \\
x<sub>n-octano</sub>= 55,2 % molar e \
y<sub>n-hexano</sub> = 41 % molar;  \\
y<sub>n-heptano</sub> = 32,6 % molar; \\
x<sub>n-octano</sub>= 26,4% molar)


***Dados***: Temperatura de bolha e de orvalho a 1,2 bar, 94 °C e 112 °C,
respectivamente.

Bibliotecas

In [2]:
import scipy.optimize as opt #otimização

Calcular a pressão de saturação com os parâmetros da equação de antoine.

\begin{equation}
   \log(P_{saturação}(bar)) = A - \frac{B}{T+C}
\end{equation}

In [3]:
def Psat(dados,T):
  n = len(dados)
  P = np.zeros(n)
  for i,j in enumerate(dados):    
    A,B,C = dados[j]
    P[i] = 10**(A - (B/(T+C)))    
  return P

In [4]:
Parametros_Antoine = {'n-hexano':[4.10014, 1170.875, 224.317],
                      'n-heptano':[4.02023, 1263.909, 216.432],
                      'n-octano':[4.05075, 1356.360, 209.635]}

pd.DataFrame(data = Parametros_Antoine,index = ["A","B","C"]).T                    

,A,B,C
n-hexano,4.10014,1170.875,224.317
n-heptano,4.02023,1263.909,216.432
n-octano,4.05075,1356.360,209.635


In [5]:
Temp = 206
for i,j in enumerate(Parametros_Antoine):
  print(f'Pressão de saturação do {j}: {round(Psat(Parametros_Antoine,Temp)[i],2)} bar a {Temp} °C')

Pressão de saturação do n-hexano: 23.94 bar a 206 °C
Pressão de saturação do n-heptano: 10.67 bar a 206 °C
Pressão de saturação do n-octano: 6.13 bar a 206 °C


Cálculo do Flash

Coeficiente de Distribuição:
\begin{equation}
  k = \frac{P_{sat}}{P}
\end{equation}
Vaporização na alimentação:
\begin{equation}
f = \frac{V}{F}
\end{equation}

Misturas multicomponentes de fases líquida e gasosa:
\begin{equation}
x_{i} = \frac{z_{i}}{1 + (K_{i} -1)\frac{V}{F}}
\end{equation} \

\begin{equation}
y_{i} = \frac{z_{i}K_i}{1 + (K_{i} -1)\frac{V}{F}}
\end{equation}



In [6]:
def flash(mistura:list,dados:dict,P,T,f):
  n = len(mistura)
  x = np.zeros(n)
  y = np.zeros(n)
  k = np.zeros(n)
   
  for i in range(0,n):
     k[i] = Psat(dados,T)[i]/P
     x[i] = mistura[i]/((1+(k[i]-1)*f))
     y[i] = x[i]*k[i]
     
  return x,y

Usando dados do exercício

In [7]:
mist = [0.3,0.32,0.38]
Parametros_Antoine = {'n-hexano':[4.10014, 1170.875, 224.317],
                      'n-heptano':[4.02023, 1263.909, 216.432],
                      'n-octano':[4.05075, 1356.360, 209.635]}

P = 1.2
T = 94 #Temperatura de bolha para estimativa inicial
f = 0.6

print(f'Componentes - Temperatura {T} °C no flash')
for i,j in enumerate(Parametros_Antoine):
  xi = flash(mist,Parametros_Antoine,P,T,f)[0][i]*100
  yi = flash(mist,Parametros_Antoine,P,T,f)[1][i]*100
  print(f'{j}: fase líquida {round(xi,2)} % | fase gasosa {round(yi,2)} %')


Componentes - Temperatura 94 °C no flash
n-hexano: fase líquida 17.44 % | fase gasosa 38.38 %
n-heptano: fase líquida 37.9 % | fase gasosa 28.07 %
n-octano: fase líquida 64.22 % | fase gasosa 20.52 %


É necessário ajustar a temperatura do flash adotando o seguinte critério: 

\begin{equation}
  \sum\limits_{i=0}^{C} z_i = \sum\limits_{i=0}^{C} y_i =\sum\limits_{i=0}^{C} x_i = 1
\end{equation} \

\begin{equation}
  \sum\limits_{i=0}^{C} \frac{z_{i}}{1 + (K_{i} -1)\frac{V}{F}}= 1 \\
    \sum\limits_{i=0}^{C} \frac{z_{i}K_i}{1 + (K_{i} -1)\frac{V}{F}} = 1
\end{equation} \





In [8]:
mist = [0.3,0.32,0.38]
Parametros_Antoine = {'n-hexano':[4.10014, 1170.875, 224.317],
                      'n-heptano':[4.02023, 1263.909, 216.432],
                      'n-octano':[4.05075, 1356.360, 209.635]}

P = 1.2

f = 0.6
def FOtimizada(T): #função de otimização de 1 parâmetro
  x = sum(flash(mist,Parametros_Antoine,P,T,f)[0])
  y = sum(flash(mist,Parametros_Antoine,P,T,f)[1])                    
  return (x - 1)**2 + (y - 1)**2 #Forçando os somatórios a darem 1

resposta = round(opt.minimize_scalar(FOtimizada).x,2)

print(f'Componentes - Temperatura {resposta} °C ajustada no flash')
for i,j in enumerate(Parametros_Antoine):
  xi = flash(mist,Parametros_Antoine,P,resposta,f)[0][i]*100
  yi = flash(mist,Parametros_Antoine,P,resposta,f)[1][i]*100
  print(f'{j}: fase líquida {round(xi,2)} % | fase gasosa {round(yi,2)} %')

Componentes - Temperatura 106.22 °C ajustada no flash
n-hexano: fase líquida 13.6 % | fase gasosa 40.93 %
n-heptano: fase líquida 30.95 % | fase gasosa 32.7 %
n-octano: fase líquida 55.44 % | fase gasosa 26.37 %
